# Dillards POS Data Association Rules Project

    Charlie Marshall
    IEMS 308
    13 February 2020

## Loading Relevant Packages

In [1]:
import os
import numpy as np
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

## Loading Relevant Data Sets from Dillards POS Data

In [2]:
deptinfo = pd.read_csv("/Users/charlesmarshall/Desktop/IEMS 308/Project2/DillardsPOS/deptinfo.csv",
                       sep = ',', names = ('Dept','Description','Zeros'))
deptinfo = deptinfo.drop('Zeros', axis = 1)

In [3]:
deptinfo.head()

,Dept,Description
0,800,CLINIQUE
1,801,LESLIE
2,1100,GARY F
3,1107,JACQUES
4,1202,CABERN


In [4]:
deptinfo.shape

(60, 2)

In [5]:
deptinfo[deptinfo['Dept'] == 6505]

,Dept,Description
41,6505,ST JOHN


In [6]:
skuinfo = pd.read_csv("/Users/charlesmarshall/Desktop/IEMS 308/Project2/DillardsPOS/skuinfo.csv",
                      sep = ',', usecols = [0,1,2,3,4,5,6,7,8,9],
                      names = ('SKU', 'Dept', 'ClassID', 'UPC', 'Style', 'Color', 'Size', 'Packsize', 'Vendor', 'Brand'),
                     dtype={'Packsize': str, 'Vendor': str})

In [7]:
skuinfo.head()

,SKU,Dept,ClassID,UPC,Style,Color,Size,Packsize,Vendor,Brand
0,3,6505,113,400000003000,00 F55KT2,WHISPERWHITE,P8EA,1,5119207,TURNBURY
1,4,8101,002,400000004000,22 615CZ4,SPEARMI,S,1,3311144,C A SPOR
2,5,7307,003,400000005000,7LBS 245-01,34 SILVER,KING,1,5510554,BEAU IDE
3,8,3404,00B,400000008000,622 F05H84,MORNING MI,2T,1,2912827,HARTSTRI
4,15,2301,004,400000015000,126 MDU461,255CAMEL,12,1,0023272,JONES/LA


In [8]:
skuinfo.shape

(1564178, 10)

In [9]:
skstinfo = pd.read_csv("/Users/charlesmarshall/Desktop/IEMS 308/Project2/DillardsPOS/skstinfo.csv",
                       sep = ',', names = ('SKU', 'Store', 'Cost', 'Retail', 'Zeros'))

In [10]:
skstinfo = skstinfo.drop('Zeros', axis = 1)

In [11]:
skstinfo.head()

,SKU,Store,Cost,Retail
0,3,102,123.36,440.0
1,3,103,123.36,440.0
2,3,104,123.36,440.0
3,3,202,123.36,440.0
4,3,203,123.36,440.0


In [12]:
skstinfo.shape

(39230146, 4)

In [13]:
strinfo = pd.read_csv("/Users/charlesmarshall/Desktop/IEMS 308/Project2/DillardsPOS/strinfo.csv",
                      sep = ',', names = ('Store','City','State','Zip','Zeros'))

In [14]:
strinfo = strinfo.drop('Zeros', axis = 1)

In [15]:
strinfo.head()

,Store,City,State,Zip
0,2,ST. PETERSBURG,FL,33710
1,3,ST. LOUIS,MO,63126
2,4,LITTLE ROCK,AR,72201
3,7,FORT WORTH,TX,76137
4,9,TEMPE,AZ,85281


In [16]:
strinfo.shape

(453, 4)

The trnsact data set is extremely large and cannot be loaded in completely. In order to combat this I only used columns which might be relevant in further stages of analysis.

In [17]:
trnsact = pd.read_csv("/Users/charlesmarshall/Desktop/IEMS 308/Project2/DillardsPOS/trnsact.csv",
                      na_filter = False, usecols = [0,1,2,3,4,5,6],
                      names = ('SKU','Store','Register','TranNum','SEQ','Date','SType'))

In [18]:
trnsact.head()

,SKU,Store,Register,TranNum,SEQ,Date,SType
0,3,202,290,1100,326708721,2005-01-18,P
1,3,202,540,2700,326708721,2005-01-29,R
2,3,303,500,2100,23702074,2004-08-18,P
3,3,709,360,500,0,2005-08-14,P
4,3,802,660,400,0,2005-08-09,P


In [19]:
trnsact.shape

(120916896, 7)

In [20]:
trnsact['Date'] = pd.to_datetime(trnsact['Date'])

In [21]:
trnsact.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120916896 entries, 0 to 120916895
Data columns (total 7 columns):
SKU         int64
Store       int64
Register    int64
TranNum     int64
SEQ         int64
Date        datetime64[ns]
SType       object
dtypes: datetime64[ns](1), int64(5), object(1)
memory usage: 6.3+ GB


## Preparing Data for Association Rules Analysis - Subsetting

Taking data from raw form to the point where it is ready to be analysed with association rules. This includes making decisions about subsetting the data. In 2005, Summer began on June 21st and ended on September 22nd. However, the data set ends on August 27th, so it does not reflect the entire summer period. Additionally, we only want to look at purchase data, not return data.

### Subset the data to only include SKUs from the Calvin Klein Department

In [22]:
deptinfo[deptinfo['Description'] == 'POLOMEN ']

,Dept,Description
25,4505,POLOMEN


In [23]:
deptID = 4505

Notice that the Deptartment number is 4505 for Calvin Klein

In [24]:
poloSKU = skuinfo[skuinfo['Dept'] == deptID]

In [25]:
poloSKU.head()

,SKU,Dept,ClassID,UPC,Style,Color,Size,Packsize,Vendor,Brand
52,210,4505,107,400000210000,9ALDN 797442,FUCHSIA/FO,160,1,5715232,POLO FAS
60,258,4505,107,400000258000,9A 797493,BANANA BEA,XXL,1,5715232,POLO FAS
71,308,4505,424,400000308000,1BR 580599,CLSC WASH,36,1,5715232,POLO FAS
83,367,4505,109,400000367000,9AME 096278,BLUE/NAVY,1654,1,5715232,POLO FAS
88,397,4505,108,400000397000,8 047338,ANDOVER HE,M,1,5715232,POLO FAS


In [26]:
# The number of unique SKUs associated with the data
len(set(poloSKU['SKU']))

142108

Subsetting trnsact data using only the Polo Men Department

In [27]:
poloData = trnsact[trnsact['SKU'].isin(skuinfo[skuinfo['Dept'] == deptID]['SKU'])]

In [28]:
poloData.head()

,SKU,Store,Register,TranNum,SEQ,Date,SType
582,210,7003,180,4000,0,2004-09-13,P
937,367,2707,370,14200,0,2004-10-01,P
938,367,3307,411,3100,0,2004-10-02,P
1618,397,303,910,6100,544100323,2004-08-21,P
5055,1165,107,31,4300,0,2004-09-05,P


In [29]:
poloData.shape

(5763323, 7)

### Subset the data to only include purchases, not returns

In [30]:
poloData = poloData[poloData['SType'] == 'P']

In [31]:
poloData.shape

(5153583, 7)

### Subsetting SKUs

There are too many SKUs (142,108 unique SKUs) to do one hot encoding. So, I am going to use the 10000 most common SKU values.

In [32]:
topSKU = pd.DataFrame(poloData['SKU'].value_counts())
topSKU.reset_index(level=0, inplace=True)
topSKU = topSKU.rename(columns = {'SKU':'count', 'index':'SKU'})

In [33]:
topSKU.head()

,SKU,count
0,6486359,8611
1,6806359,6774
2,9352339,6597
3,9432339,6314
4,6246359,6309


In [34]:
top = topSKU[:100]

In [35]:
top = top.drop('count', axis = 1)

In [36]:
top.shape

(100, 1)

In [37]:
topData = poloData[poloData['SKU'].isin(top['SKU'])]

In [38]:
topData.head()

,SKU,Store,Register,TranNum,SEQ,Date,SType
5119856,439441,102,810,200,0,2005-05-24,P
5119857,439441,102,810,1900,0,2005-02-24,P
5119858,439441,102,810,2100,0,2005-03-03,P
5119859,439441,102,810,2800,0,2005-05-28,P
5119860,439441,102,810,4900,0,2005-03-04,P


In [39]:
topData.shape

(251364, 7)

## Defining a Market Basket

The next step is to organize items purchased together into market baskets. For our purposes, a unique order will have the same Store, Register, TranNum, SEQ, and Date. A variable called orderID will be created using Store, Register, TranNum, and SEQ. If two or more items have the same orderID and Date, then they are a part of the same market basket.

In [40]:
orderID = topData['Store']+topData['Register']+topData['TranNum']+topData['SEQ']+topData['Date']

In [41]:
data = pd.concat([topData,orderID], axis=1)
data = data.rename(columns = {0:'orderID'})

In [42]:
data.head()

,SKU,Store,Register,TranNum,SEQ,Date,SType,orderID
5119856,439441,102,810,200,0,2005-05-24,P,2005-05-24 00:00:00.000001112
5119857,439441,102,810,1900,0,2005-02-24,P,2005-02-24 00:00:00.000002812
5119858,439441,102,810,2100,0,2005-03-03,P,2005-03-03 00:00:00.000003012
5119859,439441,102,810,2800,0,2005-05-28,P,2005-05-28 00:00:00.000003712
5119860,439441,102,810,4900,0,2005-03-04,P,2005-03-04 00:00:00.000005812


In [43]:
len(data['SKU'].value_counts())

100

In [44]:
# Example problem, showing that there is only one unique Market Basket when sorting by date and orderID
ex1 = data.sort_values(by = ['orderID'])
ex1.head(10)

,SKU,Store,Register,TranNum,SEQ,Date,SType,orderID
64347068,5174599,507,90,400,0,2004-08-01,P,2004-08-01 00:00:00.000000997
17936272,1589441,203,440,500,0,2004-08-01,P,2004-08-01 00:00:00.000001143
12194832,1049441,203,440,500,0,2004-08-01,P,2004-08-01 00:00:00.000001143
68048414,5471167,102,810,300,0,2004-08-01,P,2004-08-01 00:00:00.000001212
77065993,6246359,704,390,400,0,2004-08-01,P,2004-08-01 00:00:00.000001494
24080575,2108810,503,350,800,0,2004-08-01,P,2004-08-01 00:00:00.000001653
67926303,5461167,1302,220,200,0,2004-08-01,P,2004-08-01 00:00:00.000001722
120572276,9972087,1007,121,600,0,2004-08-01,P,2004-08-01 00:00:00.000001728
24080652,2108810,807,230,800,0,2004-08-01,P,2004-08-01 00:00:00.000001837
5439014,469441,1302,220,400,0,2004-08-01,P,2004-08-01 00:00:00.000001922


In [45]:
onehot = pd.get_dummies(data['SKU'], prefix = 'SKU')

In [46]:
onehot.head()

,SKU_439441,SKU_459441,SKU_469441,SKU_907523,SKU_936370,SKU_947523,SKU_992340,SKU_1049441,SKU_1069441,SKU_1072340,...,SKU_8932339,SKU_9002262,SKU_9042262,SKU_9272339,SKU_9352339,SKU_9432339,SKU_9502339,SKU_9883749,SKU_9972087,SKU_9993749
5119856,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5119857,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5119858,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5119859,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5119860,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [47]:
onehot.shape

(251364, 100)

In [48]:
df = pd.concat([data,onehot],axis=1)

In [49]:
df.head()

,SKU,Store,Register,TranNum,SEQ,Date,SType,orderID,SKU_439441,SKU_459441,...,SKU_8932339,SKU_9002262,SKU_9042262,SKU_9272339,SKU_9352339,SKU_9432339,SKU_9502339,SKU_9883749,SKU_9972087,SKU_9993749
5119856,439441,102,810,200,0,2005-05-24,P,2005-05-24 00:00:00.000001112,1,0,...,0,0,0,0,0,0,0,0,0,0
5119857,439441,102,810,1900,0,2005-02-24,P,2005-02-24 00:00:00.000002812,1,0,...,0,0,0,0,0,0,0,0,0,0
5119858,439441,102,810,2100,0,2005-03-03,P,2005-03-03 00:00:00.000003012,1,0,...,0,0,0,0,0,0,0,0,0,0
5119859,439441,102,810,2800,0,2005-05-28,P,2005-05-28 00:00:00.000003712,1,0,...,0,0,0,0,0,0,0,0,0,0
5119860,439441,102,810,4900,0,2005-03-04,P,2005-03-04 00:00:00.000005812,1,0,...,0,0,0,0,0,0,0,0,0,0


In [50]:
df.drop(['SKU','Store','Register','TranNum','SEQ','Date','SType'],axis=1, inplace=True)

In [51]:
df.shape

(251364, 101)

In [52]:
df.head()

,orderID,SKU_439441,SKU_459441,SKU_469441,SKU_907523,SKU_936370,SKU_947523,SKU_992340,SKU_1049441,SKU_1069441,...,SKU_8932339,SKU_9002262,SKU_9042262,SKU_9272339,SKU_9352339,SKU_9432339,SKU_9502339,SKU_9883749,SKU_9972087,SKU_9993749
5119856,2005-05-24 00:00:00.000001112,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5119857,2005-02-24 00:00:00.000002812,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5119858,2005-03-03 00:00:00.000003012,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5119859,2005-05-28 00:00:00.000003712,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5119860,2005-03-04 00:00:00.000005812,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [53]:
finalData = df.groupby(['orderID']).sum()

In [54]:
finalData.head()

,SKU_439441,SKU_459441,SKU_469441,SKU_907523,SKU_936370,SKU_947523,SKU_992340,SKU_1049441,SKU_1069441,SKU_1072340,...,SKU_8932339,SKU_9002262,SKU_9042262,SKU_9272339,SKU_9352339,SKU_9432339,SKU_9502339,SKU_9883749,SKU_9972087,SKU_9993749
orderID,,,,,,,,,,,,,,,,,,,,,
2004-08-01 00:00:00.000000997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2004-08-01 00:00:00.000001143,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2004-08-01 00:00:00.000001212,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2004-08-01 00:00:00.000001494,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2004-08-01 00:00:00.000001653,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [55]:
finalData.shape

(210929, 100)

In [56]:
final = finalData >= 1

In [57]:
a = final.sum()
b = a/886008

In [58]:
b = pd.DataFrame(b)
b.reset_index(level=0, inplace=True)
b = b.rename(columns = {0:'prob', 'index':'SKU'})

In [59]:
b.head()

,SKU,prob
0,SKU_439441,0.002167
1,SKU_459441,0.002328
2,SKU_469441,0.001938
3,SKU_907523,0.002155
4,SKU_936370,0.002196


In [60]:
b.sort_values(by = 'prob',ascending=False)

,SKU,prob
73,SKU_6486359,0.009614
76,SKU_6806359,0.007550
94,SKU_9352339,0.007144
69,SKU_6246359,0.007060
95,SKU_9432339,0.006795
9,SKU_1072340,0.004977
11,SKU_1152340,0.004972
93,SKU_9272339,0.004219
23,SKU_3269431,0.003604
39,SKU_5228351,0.003433


In [76]:
frequentItemsets = apriori(final, min_support=0.0001, use_colnames=True)

In [77]:
frequentItemsets

,support,itemsets
0,0.009103,(SKU_439441)
1,0.009781,(SKU_459441)
2,0.008140,(SKU_469441)
3,0.009050,(SKU_907523)
4,0.009226,(SKU_936370)
5,0.008145,(SKU_947523)
6,0.012720,(SKU_992340)
7,0.009193,(SKU_1049441)
8,0.010018,(SKU_1069441)
9,0.020908,(SKU_1072340)


In [78]:
rules = association_rules(frequentItemsets, metric="lift", min_threshold=1)

In [79]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(SKU_1049441),(SKU_439441),0.009193,0.009103,0.001721,0.187210,20.566665,0.001637,1.219131
1,(SKU_439441),(SKU_1049441),0.009103,0.009193,0.001721,0.189062,20.566665,0.001637,1.221805
2,(SKU_439441),(SKU_1069441),0.009103,0.010018,0.000104,0.011458,1.143821,0.000013,1.001457
3,(SKU_1069441),(SKU_439441),0.010018,0.009103,0.000104,0.010412,1.143821,0.000013,1.001323
4,(SKU_439441),(SKU_1589441),0.009103,0.008207,0.001375,0.151042,18.405007,0.001300,1.168247
5,(SKU_1589441),(SKU_439441),0.008207,0.009103,0.001375,0.167533,18.405007,0.001300,1.190315
6,(SKU_439441),(SKU_3249431),0.009103,0.012777,0.000137,0.015104,1.182155,0.000021,1.002363
7,(SKU_3249431),(SKU_439441),0.012777,0.009103,0.000137,0.010761,1.182155,0.000021,1.001676
8,(SKU_439441),(SKU_6119446),0.009103,0.011990,0.000114,0.012500,1.042551,0.000005,1.000517
9,(SKU_6119446),(SKU_439441),0.011990,0.009103,0.000114,0.009490,1.042551,0.000005,1.000391


In [80]:
rules.describe()

,antecedent support,consequent support,support,confidence,lift,leverage,conviction
count,438.000000,438.000000,438.000000,438.000000,438.000000,438.000000,438.000000
mean,0.010345,0.010345,0.000471,0.073210,9.391543,0.000370,1.076387
std,0.007187,0.007187,0.000512,0.078111,9.109040,0.000463,0.102819
min,0.000318,0.000318,0.000104,0.002935,1.015415,0.000002,1.000153
25%,0.008363,0.008363,0.000129,0.015443,2.189526,0.000100,1.008498
50%,0.009297,0.009297,0.000251,0.038598,5.583817,0.000177,1.028905
75%,0.011837,0.011837,0.000531,0.107968,14.065761,0.000450,1.106857
max,0.040383,0.040383,0.002423,0.344828,37.511159,0.002212,1.512285


In [81]:
rules[ rules['lift'] >= 10 ]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(SKU_1049441),(SKU_439441),0.009193,0.009103,0.001721,0.187210,20.566665,0.001637,1.219131
1,(SKU_439441),(SKU_1049441),0.009103,0.009193,0.001721,0.189062,20.566665,0.001637,1.221805
4,(SKU_439441),(SKU_1589441),0.009103,0.008207,0.001375,0.151042,18.405007,0.001300,1.168247
5,(SKU_1589441),(SKU_439441),0.008207,0.009103,0.001375,0.167533,18.405007,0.001300,1.190315
10,(SKU_459441),(SKU_1069441),0.009781,0.010018,0.001911,0.195347,19.500359,0.001813,1.230322
11,(SKU_1069441),(SKU_459441),0.010018,0.009781,0.001911,0.190724,19.500359,0.001813,1.223587
12,(SKU_1869441),(SKU_459441),0.008652,0.009781,0.001550,0.179178,18.319852,0.001466,1.206376
13,(SKU_459441),(SKU_1869441),0.009781,0.008652,0.001550,0.158507,18.319852,0.001466,1.178082
20,(SKU_469441),(SKU_1079441),0.008140,0.007951,0.001607,0.197437,24.833197,0.001542,1.236102
21,(SKU_1079441),(SKU_469441),0.007951,0.008140,0.001607,0.202147,24.833197,0.001542,1.243161


In [82]:
rules[ rules['lift'] >= 10 ].describe()

,antecedent support,consequent support,support,confidence,lift,leverage,conviction
count,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000
mean,0.007010,0.007010,0.000593,0.131224,20.411443,0.000556,1.157795
std,0.005382,0.005382,0.000676,0.098502,7.303378,0.000630,0.132768
min,0.000318,0.000318,0.000104,0.004153,10.197491,0.000095,1.003813
25%,0.001025,0.001025,0.000119,0.021670,14.276482,0.000110,1.020857
50%,0.008605,0.008605,0.000209,0.139735,19.500359,0.000200,1.152392
75%,0.009805,0.009805,0.001005,0.198432,25.308696,0.000936,1.235138
max,0.030010,0.030010,0.002423,0.344828,37.511159,0.002212,1.512285


In [108]:
rules[(rules['lift'] >= 10) & (rules['confidence'] >= 0.25) ]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
288,"(SKU_1049441, SKU_439441)",(SKU_1589441),0.001721,0.008207,0.000474,0.275482,33.568551,0.000460,1.368901
289,"(SKU_1049441, SKU_1589441)",(SKU_439441),0.001664,0.009103,0.000474,0.284900,31.298819,0.000459,1.385677
290,"(SKU_439441, SKU_1589441)",(SKU_1049441),0.001375,0.009193,0.000474,0.344828,37.511159,0.000461,1.512285
294,"(SKU_1869441, SKU_459441)",(SKU_1069441),0.001550,0.010018,0.000493,0.318043,31.748439,0.000478,1.451678
295,"(SKU_459441, SKU_1069441)",(SKU_1869441),0.001911,0.008652,0.000493,0.258065,29.826460,0.000477,1.336164
296,"(SKU_1869441, SKU_1069441)",(SKU_459441),0.001930,0.009781,0.000493,0.255528,26.126185,0.000474,1.330097
300,"(SKU_1072340, SKU_6447365)",(SKU_9352339),0.000768,0.030010,0.000247,0.320988,10.695988,0.000223,1.428531
319,"(SKU_2078810, SKU_1571243)",(SKU_5901243),0.000318,0.009273,0.000104,0.328358,35.409135,0.000101,1.475082
336,"(SKU_4912330, SKU_5901243)",(SKU_1571243),0.000773,0.009297,0.000251,0.325153,34.974134,0.000244,1.468042
337,"(SKU_4912330, SKU_1571243)",(SKU_5901243),0.000749,0.009273,0.000251,0.335443,36.173141,0.000244,1.490808


In [95]:
rules[rules['lift'] >= 37.511159]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
290,"(SKU_439441, SKU_1589441)",(SKU_1049441),0.001375,0.009193,0.000474,0.344828,37.511159,0.000461,1.512285
291,(SKU_1049441),"(SKU_439441, SKU_1589441)",0.009193,0.001375,0.000474,0.051573,37.511159,0.000461,1.052928


In [96]:
rules[rules['lift'] >= 30]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
288,"(SKU_1049441, SKU_439441)",(SKU_1589441),0.001721,0.008207,0.000474,0.275482,33.568551,0.000460,1.368901
289,"(SKU_1049441, SKU_1589441)",(SKU_439441),0.001664,0.009103,0.000474,0.284900,31.298819,0.000459,1.385677
290,"(SKU_439441, SKU_1589441)",(SKU_1049441),0.001375,0.009193,0.000474,0.344828,37.511159,0.000461,1.512285
291,(SKU_1049441),"(SKU_439441, SKU_1589441)",0.009193,0.001375,0.000474,0.051573,37.511159,0.000461,1.052928
292,(SKU_439441),"(SKU_1049441, SKU_1589441)",0.009103,0.001664,0.000474,0.052083,31.298819,0.000459,1.053190
293,(SKU_1589441),"(SKU_1049441, SKU_439441)",0.008207,0.001721,0.000474,0.057770,33.568551,0.000460,1.059486
294,"(SKU_1869441, SKU_459441)",(SKU_1069441),0.001550,0.010018,0.000493,0.318043,31.748439,0.000478,1.451678
299,(SKU_1069441),"(SKU_1869441, SKU_459441)",0.010018,0.001550,0.000493,0.049219,31.748439,0.000478,1.050137
319,"(SKU_2078810, SKU_1571243)",(SKU_5901243),0.000318,0.009273,0.000104,0.328358,35.409135,0.000101,1.475082
322,(SKU_5901243),"(SKU_2078810, SKU_1571243)",0.009273,0.000318,0.000104,0.011247,35.409135,0.000101,1.011054


In [88]:
skuinfo[skuinfo['SKU'].isin([1049441,1589441,439441])]

,SKU,Dept,ClassID,UPC,Style,Color,Size,Packsize,Vendor,Brand
68584,439441,4505,404,400009441043,2BAO 581965,NAVY,34,1,5745232,POLO FAS
163747,1049441,4505,404,400009441104,7BAO 581972,UNFR KHAKI,34,1,5745232,POLO FAS
248103,1589441,4505,404,400009441158,3BAO 581965,SAND,34,1,5745232,POLO FAS


In [89]:
skuinfo[skuinfo['SKU'].isin([1869441,459441,1069441])]

,SKU,Dept,ClassID,UPC,Style,Color,Size,Packsize,Vendor,Brand
71637,459441,4505,404,400009441045,2BAO 581965,NAVY,36,1,5745232,POLO FAS
166877,1069441,4505,404,400009441106,7BAO 581972,UNFR KHAKI,36,1,5745232,POLO FAS
291253,1869441,4505,404,400009441186,3BAO 581965,SAND,36,1,5745232,POLO FAS


In [90]:
skuinfo[skuinfo['SKU'].isin([1072340,6447365,9352339])]

,SKU,Dept,ClassID,UPC,Style,Color,Size,Packsize,Vendor,Brand
167551,1072340,4505,216,400002340107,9 430070,BLACK,L,1,5745232,POLO FAS
1009867,6447365,4505,216,400007365644,1 431002,INK,L,1,5745232,POLO FAS
1463779,9352339,4505,216,400002339935,0 430071,WHITE,L,1,5745232,POLO FAS


In [91]:
skuinfo[skuinfo['SKU'].isin([2078810,1571243,5901243,4912330,9972087])]

,SKU,Dept,ClassID,UPC,Style,Color,Size,Packsize,Vendor,Brand
245379,1571243,4505,302,400001243157,3VPP 650185,SPA BLUE,ONE,1,5715232,POLO FAS
323397,2078810,4505,302,400008810207,5VPP 651649,BSC WHITE,ONE,1,5715232,POLO FAS
769457,4912330,4505,302,400002330491,3VPP 656523,VALOR RED,ONE,1,5715232,POLO FAS
925021,5901243,4505,302,400001243590,7VPP 656475,OASIS YELL,ONE,1,5715232,POLO FAS
1560073,9972087,4505,302,400002087997,2VPP 650100,RELAY BLUE,ONE,1,5715232,POLO FAS


In [92]:
skuinfo[skuinfo['SKU'].isin([5709431,3269431,6149446])]

,SKU,Dept,ClassID,UPC,Style,Color,Size,Packsize,Vendor,Brand
510957,3269431,4505,404,400009431326,3BRG 581965,SAND,36,1,5745232,POLO FAS
894494,5709431,4505,404,400009431570,2BRG 581965,NAVY,36,1,5745232,POLO FAS
963876,6149446,4505,404,400009446614,7BRG 581972,UNFR KHAKI,36,1,5745232,POLO FAS


In [93]:
skuinfo[skuinfo['SKU'].isin([4898351,5228351,5549233,5748351])]

,SKU,Dept,ClassID,UPC,Style,Color,Size,Packsize,Vendor,Brand
767035,4898351,4505,107,400008351489,2AEB 793365,CLSC BLUE,L,1,5745232,POLO FAS
818845,5228351,4505,107,400008351522,5AEB 793365,CLSC WHITE,L,1,5745232,POLO FAS
869386,5549233,4505,107,400009233554,6AEB 790003,W/W INDIGO,L,1,5745232,POLO FAS
900529,5748351,4505,107,400008351574,7AEB 793365,BLU/WHT STR,L,1,5745232,POLO FAS


In [94]:
skuinfo[skuinfo['SKU'].isin([6457365,9432339,8613189])]

,SKU,Dept,ClassID,UPC,Style,Color,Size,Packsize,Vendor,Brand
1011459,6457365,4505,216,400007365645,1 431002,INK,XL,1,5745232,POLO FAS
1348989,8613189,4505,216,400003189861,9 435866,WICKET YEL,XL,1,5745232,POLO FAS
1476135,9432339,4505,216,400002339943,0 430071,WHITE,XL,1,5745232,POLO FAS
